# Metody kryptografii w analizie danych

# Algorytm RSA

Obecnie najpopularniejszy algorytm kryptografi asymetrycznej. Opublikowany w 1977 roku w *Scientific American's Mathematical Games* przez Rona Rivesta, Adiego Shamira i Leonarda Adlemana. Opiera się na problemie faktoryzacji liczb.

W uproszczonej wersji algorytm RSA wygląda następująco:

### Generowanie kluczy

- Wybieramy dwie losowe liczby pierwsze $p$ i $q$
- Obliczamy $n=pq$
- Obliczamy funkcję Eulera $\phi(n)=(p-1)(q-1)$
- Wybieramy $e\in\{2,...,\phi(n)-1\}$ względnie pierwszą z $\phi(n)$
- Obliczamy $d=e^{-1}\mod\phi(n)$
- Klucz publiczny to para $(n,e)$ a klucz prywatny to para $(n,d)$.


### Szyfrowanie
Mamy dany klucz publiczny $(n,e)$. Wiadomość szyfrowana jest liczbą $m\in [0,n)$.
- Obliczamy szyfrogram $$c=m^e\mod n.$$

Teoretycznie może się zdarzyć, że szyfrogram $c$ jest taki sam, jak szyfrowana wiadomość $m$.

### Deszyfrowanie
- Obliczamy odszyfrowaną wiadomość $$m=c^d=m^{ed}\mod n$$

## Ćwiczenie 1.

Zaimplementuj uproszczony algorytm RSA.

In [101]:
from Crypto.Util import number
import random
from math import gcd

MIN_DIST = 2**64
MIN_E = 2**16

def gen_keys(n_length=2048):
    min_dist = 2 ** int(0.1 * n_length) + 1
    min_e = 2 ** int(0.05 * n_length) + 1
    p = number.getPrime(n_length)
    q = number.getPrime(n_length)
    while abs(p - q) < min_dist:
        q = number.getPrime(n_length)
    n = p * q
    euler_n = (p - 1) * (q - 1)
    e = random.randrange(min_e, euler_n)
    while gcd(e, euler_n) != 1:
        e = random.randrange(2, euler_n)
    d = pow(e, -1, euler_n)
    return (n, e), (n, d)

In [102]:
def rsa_encrypt(message, public_key):
    (n, e) = public_key
    return pow(message, e, n)


def rsa_decrypt(cipher, private_key):
    (n, d) = private_key
    return pow(cipher, d, n)

In [95]:
pub_key, priv_key = gen_keys(1024)

In [96]:
message = 2137
ciphertext = rsa_encrypt(message, pub_key)
rsa_decrypt(ciphertext, priv_key)

2137

# Proste ataki na RSA

## Brute force

Najprostszym (ale też najmniej efektywnym i najbardziej bezsensownym) możliwym atakiem na RSA jest atak brute force:

#### Krok 1.
- Wersja 1: tablicujemy wszystkie liczby pierwsze mniejsze lub równe $\sqrt{n}$ do tablicy $T$.
- Wersja 2: pomijamy tablicowanie i w kroku 2. po prostu sprawdzamy wszystkie liczby naturalne mniejsze lub równe $\sqrt{n}$

#### Krok 2.
- Wersja 1: sprawdzamy podzielność $n$ przez każdą $p\in T$.
- Wersja 2: sprawdzamy podzielność $n$ przez każdą liczbę naturalną mniejszą lub równą $\sqrt{n}$

#### Krok 3.
Mając faktoryzację $pq=n$ obliczamy $\phi(n)$ oraz odtwarzamy $d=e^{-1}\mod \phi(n)$.

## Ćwiczenie 2.
Zaimplementuj atak brute force i przetestuj go na swojej implementacji RSA. Jeżeli atak zakończył się sukcesem - popraw implementację kryptosystemu. Jakie założenia powinny spełniać parametry kryptosystemu, aby atak się nie powiódł?

In [19]:
import math

def sieve(num):
    prime = [True for _ in range(num + 1)]
    p = 2
    while p * p <= num:
        # If prime[p] is not
        # changed, then it is a prime
        if prime[p]:
            # Updating all multiples of p
            for i in range(p * p, num + 1, p):
                prime[i] = False
        p += 1
    return prime

In [103]:
(n, e), _ = gen_keys(15)
t = sieve(int(math.sqrt(n)))
primes = [i for i, n in enumerate(t) if n and i > 1]

In [85]:
for can in primes:
    if n % can == 0:
        print(f"Broken, p = {can}, q = {n // can}")

## Faktoryzacja Fermata

Faktoryzacja Fermata liczby nieparzystej $n$ opiera się na znalezieniu pary liczb $a,b$ takich, że $n=a^2-b^2$. Wtedy od razu otrzymujemy faktoryzację $$n=(a+b)(a-b).$$

Dla dowolnej złożonej liczby nieparzystej (tzn. liczby nieparzystej nie będącej liczbą pierwszą) znajdziemy takie liczby $a,b$. W szczególności, w przypadku $n=pq$ (jak w RSA) mamy $$a=\frac{p+q}{2},\qquad b=\frac{p-q}{2}.$$

#### Krok 0.
Sprawdzamy, czy $\sqrt{n}$ jest liczbą naturalną. Jeżeli tak - znaleźliśmy faktoryzację i koniec algorytmu.
#### Krok 1.
Definiujemy zmienne pomocnicze
$$a = \left\lceil\sqrt{n}\right\rceil,\qquad
b^2 = a^2 - n.$$
#### Krok 2.
Jeżeli $\sqrt{b^2}$ jest liczbą całkowitą, to kończymy algorytm i zwracamy $a$ oraz $b=\sqrt{b^2}$. Jeżeli nie, to aktualizujemy zmienne:
$$a=a + 1,\qquad b^2=a^2 - n.$$


## Ćwiczenie 3.
Zaimplementuj atak z użyciem faktoryzacji Fermata i przetestuj go na swojej implementacji RSA. Jeżeli atak zakończył się sukcesem - popraw implementację kryptosystemu. Jakie założenia powinny spełniać parametry kryptosystemu, aby atak się nie powiódł?

In [62]:
# TYPE YOUR CODE BELOW
def is_square(apositiveint):
  x = apositiveint // 2
  seen = set([x])
  while x * x != apositiveint:
    x = (x + (apositiveint // x)) // 2
    if x in seen: return False
    seen.add(x)
  return True

In [77]:
def fermat_attack(n, max_iters, log = False):
    if is_square(n):
        print(f"Broken: p and q = {round(math.sqrt(n))}")
    a = math.ceil(math.sqrt(n))
    b_sqr = a ** 2 - n
    for _ in range(max_iters) :
        if is_square(b_sqr):
            b = n // a
            p = a + b
            q = a - b
            print(f"Broken: p = {p} and q = {q}")
            return True
        a += 1
        b_sqr = a ** 2 - n
    if log:
        print(f"Not broken in {max_iters} iters")
    return False

In [75]:
(n, e), _ = gen_keys(35)
print(f"n = {n}")
fermat_attack(n, 1000000)

n = 483842851609230329311
Not broken in 1000000 iters


In [107]:
for _ in range(1000):
    (n, e), _ = gen_keys(35)
    fermat_attack(n, 1000, False)

# Ataki nie wykorzystujące faktoryzacji

## Atak naiwny

Jeżeli wykładnik $e$ jest niewielki oraz dla wiadomości $m$ wartość $m^e<n$, to $c=m^e\mod n=m^e$. Możemy zatem odzyskać $m$ obliczając $m=\sqrt[e]{c}$.

## Ćwiczenie 3.
Sprawdź, czy Twoja implementacja RSA jest podatna na powyższy atak wykonując co najmniej 1000 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

In [106]:
# TYPE YOUR CODE BELOW

MESSAGE = 1337
for _ in range(1000):
    public_key, _ = gen_keys(35)
    n, e = public_key
    c = rsa_encrypt(MESSAGE, public_key)
    if pow(c, 1 / e) == MESSAGE:
        print("Message too short")
     

## Atak Wienera

Atak Wienera odtwarza klucz prywatny $d$ z klucza publicznego $(n,e)$ bez konieczności faktoryzacji $n$.

#### Krok 1.
Przeksztacamy ułamek $\frac{e}{n}$ na ułamek łańcuchowy postaci $${\displaystyle x=a_{0}+{\cfrac {1}{a_{1}+{\cfrac {1}{a_{2}+{\cfrac {1}{a_{3}+{\cfrac {1}{a_{4}+\ddots \,\frac{1}{a_k}}}}}}}}}}$$o standardowej reprezentacji $[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$.

#### Krok 2.
Dla każdego reduktu ułamka łańcuchowego, tzn. ułamka łańcuchowego postaci $[a_0]$, $[a_0;a_1]$, $[a_0;a_1,a_2]$,...,$[a_0;a_1,a_2,...,a_{k−2},a_{k−1},a_k]$ sprawdzamy, czy jest on rozwinięciem ułamka $\frac{s}{d}$ w ułamek łańcuchowy dla pewnej stałej $s$:
- niech $l$ oznacza licznik a $m$ - mianownik reduktu
- sprawdź, czy mianownik $m$ jest nieparzysty. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- sprawdź, czy $em=1\mod l$. Jeżeli nie - przejdź do sprawdzania kolejnego reduktu.
- zdefiniuj wielomian pomocniczy $$Q(x)=x^2-(n-\frac{em-1}{l}+1)x+n.$$Jeżeli pierwiastki tego wielomianu są liczbami całkowitymi, to aktualny mianownik $m$ reduktu jest szukanym kluczem prywatnym $d$.


Atak Wienera niekoniecznie musi zakończyć się sukcesem - jeżeli sprawdzimy wszystkie redukty i żaden nie jest rozwinięciem $\frac{s}{d}$, to dany zestaw parametrów RSA jest odporny na ten atak.

## Ćwiczenie 4.

Sprawdź, czy Twoja implementacja RSA jest podatna na atak Wienera wykonując co najmniej 100 prób. Jeżeli którakolwiek zakończyła się powodzeniem - popraw implementację. Jakie założenia powinny spełniać parametry kryptosystemu, żeby atak nie miał szans powodzenia?

In [ ]:
# TYPE YOUR CODE BELOW
